In [ ]:
# --- Notebook Cell 1: Imports and Merge Functions ---
import os
import pandas as pd

In [ ]:
def load_and_merge_folder(folder_path, date_col="date"):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
    merged_df = None
    merged_columns = set()

    for filename in csv_files:
        df = pd.read_csv(os.path.join(folder_path, filename))
        df.columns = [col.lower().strip() for col in df.columns]

        if date_col not in df.columns:
            print(f"Skipping {filename} — no `{date_col}` column")
            continue

        df[date_col] = pd.to_datetime(df[date_col])
        new_cols = [col for col in df.columns if col == date_col or col not in merged_columns]

        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df[new_cols], on=date_col, how="outer")

        merged_columns.update(new_cols)

    return merged_df

In [17]:
# --- Notebook Cell 2: Clean & Interpolate ---
def interpolate_and_clean(df, date_col="date"):
    df = df.sort_values(date_col)
    numeric_cols = df.select_dtypes(include="number").columns
    df[numeric_cols] = df[numeric_cols].interpolate(method="linear", limit_direction="both")
    df = df.dropna(axis=1, how="all")
    df = df.dropna(axis=0, how="any")
    return df

In [19]:
# --- Notebook Cell 3: Merge Helpers ---
def merge_on_date(df1, df2, date_col="date"):
    shared_cols = set(df1.columns).intersection(df2.columns) - {date_col}
    df2 = df2.drop(columns=shared_cols)
    return pd.merge(df1, df2, on=date_col, how="outer")

In [21]:
# --- Notebook Cell 4: Full Pipeline Execution ---
# Merge all FRED-style CSVs from folder
fred_df = load_and_merge_folder("fredstuff/")
fred_df = interpolate_and_clean(fred_df)

In [23]:
fred_df.head()

,date,layman__oil_price,layman__gas_price,layman__mortgage_rate,layman__housing_prices,layman__grocery_index,layman__retail_sales,layman__electricity_cost_index,layman__used_car_prices,layman__minimum_wage_trend,...,linchpin__nonfarm_payrolls,linchpin__real_gdp_growth,linchpin__s&p500_returns,linchpin__ism_nonmanufacturing_index,linchpin__export_prices,google_trends_unemployment,tiktok_recession_hashtag_velocity,reddit_finance_sentiment,twitter_macro_fear_score,news_coverage_volatility_index
0,1997-01-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,131011.0,13878.147,-0.020108,0.006353,-0.003048,10.0,0.0,54.0,17.0,30.0
1,1997-02-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,131011.0,13878.147,-0.020108,0.006353,-0.003048,10.0,0.0,54.0,17.0,30.0
2,1997-03-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,131011.0,13878.147,-0.020108,0.006353,-0.003048,10.0,0.0,54.0,17.0,30.0
3,1997-04-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,131011.0,13878.147,-0.020108,0.006353,-0.003048,10.0,0.0,54.0,17.0,30.0
4,1997-05-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,131011.0,13878.147,-0.020108,0.006353,-0.003048,10.0,0.0,54.0,17.0,30.0


In [25]:
# Load and clean stock data
stock_df = pd.read_csv("unified_stock_data.csv")
stock_df.rename(columns={"Date": "date"}, inplace=True)
stock_df['date'] = pd.to_datetime(stock_df['date'])

In [27]:
stock_df.head()

,date,Open_CSCO,High_CSCO,Low_CSCO,Close_CSCO,Adj Close_CSCO,Volume_CSCO,Open_ADBE,High_ADBE,Low_ADBE,...,Low_FORD,Close_FORD,Adj Close_FORD,Volume_FORD,Open_AAPL,High_AAPL,Low_AAPL,Close_AAPL,Adj Close_AAPL,Volume_AAPL
0,2000-01-03,54.96875,55.1250,51.78125,54.03125,35.572937,53076000,16.812500,16.875000,16.062500,...,28.125,30.625,30.625,4260,0.936384,1.004464,0.907924,0.999442,0.842151,535796800
1,2000-01-04,52.75000,53.5000,50.87500,51.00000,33.577236,50805600,15.750000,16.453125,14.984375,...,26.250,30.625,30.625,2360,0.966518,0.987723,0.903460,0.915179,0.771149,512377600
2,2000-01-05,50.03125,52.2500,48.62500,50.84375,33.474365,68524000,14.562500,15.687500,14.562500,...,26.250,28.750,28.750,720,0.926339,0.987165,0.919643,0.928571,0.782433,778321600
3,2000-01-06,50.40625,50.9375,49.31250,50.00000,32.918861,48242600,15.359375,15.656250,15.156250,...,26.250,27.500,27.500,4540,0.947545,0.955357,0.848214,0.848214,0.714723,767972800
4,2000-01-07,49.68750,53.0000,49.68750,52.93750,34.852848,62260600,15.390625,16.187500,15.218750,...,26.250,28.750,28.750,4330,0.861607,0.901786,0.852679,0.888393,0.748578,460734400


In [33]:
# Load and clean financial data
financial_df = pd.read_csv("financial_data_cleaned2.csv")
financial_df.rename(columns={"Date": "date"}, inplace=True)
financial_df['date'] = pd.to_datetime(financial_df['date'])

In [35]:
financial_df.head()

,date,Adj Close_^GSPC,Adj Close_^IXIC,Adj Close_^VIX,Bond Yields,Inflation,Unemployment,Interest Rate,Consumer Sentiment,GDP,...,Interest Rate_lag30_lag90_lag180_rolling90_rolling180,Consumer Sentiment_lag30_lag90_lag180_rolling90_rolling180,GDP_lag30_lag90_lag180_rolling90_rolling180,Rolling_Std_30d,market crash,buy gold,recession,inflation,stock market crash,interest rates
0,2004-01-01,1111.920044,2003.369995,18.309999,4.27,186.3,5.7,1.0,103.8,11923.447,...,1.746524,91.359951,10880.208017,20.100536,31.0,28.0,4.0,49.0,7.0,61.0
1,2004-01-02,1108.479980,2006.680054,18.219999,4.38,186.3,5.7,1.0,103.8,11923.447,...,1.745816,91.332796,10881.780753,19.347632,31.0,28.0,4.0,49.0,7.0,61.0
2,2004-01-05,1122.219971,2047.359985,17.490000,4.41,186.3,5.7,1.0,103.8,11923.447,...,1.745106,91.304673,10883.350672,20.203467,31.0,28.0,4.0,49.0,7.0,61.0
3,2004-01-06,1123.670044,2057.370117,16.730000,4.29,186.3,5.7,1.0,103.8,11923.447,...,1.744394,91.275580,10884.917773,20.939856,31.0,28.0,4.0,49.0,7.0,61.0
4,2004-01-07,1126.329956,2077.679932,15.500000,4.27,186.3,5.7,1.0,103.8,11923.447,...,1.743681,91.245519,10886.482057,21.774041,31.0,28.0,4.0,49.0,7.0,61.0


In [37]:
# Merge all sources
merged = merge_on_date(fred_df, stock_df)
merged = merge_on_date(merged, financial_df)

In [39]:
merged.head()

,date,layman__oil_price,layman__gas_price,layman__mortgage_rate,layman__housing_prices,layman__grocery_index,layman__retail_sales,layman__electricity_cost_index,layman__used_car_prices,layman__minimum_wage_trend,...,Interest Rate_lag30_lag90_lag180_rolling90_rolling180,Consumer Sentiment_lag30_lag90_lag180_rolling90_rolling180,GDP_lag30_lag90_lag180_rolling90_rolling180,Rolling_Std_30d,market crash,buy gold,recession,inflation,stock market crash,interest rates
0,1997-01-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997-02-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997-03-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997-04-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997-05-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# --- Add mutual fund NAV pivoted data ---
mutual_df = pd.read_csv("cleaned_mutual_fund_nav copy.csv")
mutual_df.columns = [col.strip() for col in mutual_df.columns]
mutual_df['NAV__Scheme Code'] = mutual_df['NAV__Scheme Code'].astype(str)
mutual_df['date'] = pd.to_datetime(mutual_df['date'])

mf_pivot = mutual_df.pivot(index='date', columns='NAV__Scheme Code', values='NAV__Net Asset Value')
mf_pivot.columns = [f"nav_{code}" for code in mf_pivot.columns]
mf_pivot = mf_pivot.reset_index()

merged = merge_on_date(merged, mf_pivot)
merged = interpolate_and_clean(merged)

In [43]:
merged.head()

,date,layman__oil_price,layman__gas_price,layman__mortgage_rate,layman__housing_prices,layman__grocery_index,layman__retail_sales,layman__electricity_cost_index,layman__used_car_prices,layman__minimum_wage_trend,...,nav_153434,nav_153435,nav_153436,nav_153438,nav_153441,nav_153442,nav_153443,nav_153444,nav_153445,nav_153451
0,1997-01-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085
1,1997-02-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085
2,1997-03-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085
3,1997-04-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085
4,1997-05-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085


In [51]:
# Save to disk
# merged.to_csv("final_merged_dataset.csv", index=False)
merged.to_parquet("final_merged_dataset.parquet", index=False)
print("Final dataset saved to final_merged_dataset.parquet")

Final dataset saved to final_merged_dataset.parquet


In [49]:
merged.head()

,date,layman__oil_price,layman__gas_price,layman__mortgage_rate,layman__housing_prices,layman__grocery_index,layman__retail_sales,layman__electricity_cost_index,layman__used_car_prices,layman__minimum_wage_trend,...,nav_153434,nav_153435,nav_153436,nav_153438,nav_153441,nav_153442,nav_153443,nav_153444,nav_153445,nav_153451
0,1997-01-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085
1,1997-02-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085
2,1997-03-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085
3,1997-04-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085
4,1997-05-01,27.65,1.316,8.25,100.0,166.3,268044.0,117.3,153.9,5.15,...,9.8,35.1732,10.0213,1002.237,10.019,10.0197,10.083,10.0832,10.0819,1002.7085
